# Spark Database Workloads
Tested only with __postgresql__

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


## Instantiate Classes

In [2]:
import os
import sys

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkDBwls as db

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    db = importlib.reload(db)

__desc__ = "read and write files from and to postgresql database"
clsSDB = db.SQLWorkLoads(desc=__desc__)
if clsSDB.session:
    clsSDB._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
23/03/27 14:25:36 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/03/27 14:25:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/27 14:25:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/27 14:25:37 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/03/27 14:25:37 WARN FileSys

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).



read and write files from and to postgresql database class initialization and load complete!


## Load data from DB using SQL query

In [3]:
_from_date = '2022-01-05'
_to_date = '2022-01-05'
_asset = 'axial'

# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date >= '{_from_date}' and "+\
#         f"mcap_date <= '{_to_date}'"
_query = "SELECT * FROM warehouse.mcap_past wmp "+\
        f"WHERE deactivate_dt IS NULL AND wmp.mcap_value > 1000000 "+\
        f"AND wmp.mcap_date BETWEEN '{_from_date}' AND '{_to_date}' "+\
        f"AND wmp.asset_name = '{_asset}' "
#         f"AND wmp.asset_name = '{_asset}' AND uuid='{_uuid}'"


_mcap_sdb=clsSDB.read_data_from_table(
    select=_query,
    db_table="",
    db_column="",
    lower_bound=None,
    upper_bound=None,
#     **_kwargs
)

if _mcap_sdb.count() > 0:
    print(_mcap_sdb.show(n=3,vertical=True))
else:
    print("Returned empty data set")

-RECORD 0-----------------------------
 mcap_past_pk  | 66022                
 uuid          | 6396fb727cc473c58... 
 data_source   | coingecko            
 asset_name    | axial                
 asset_symbol  | axial                
 mcap_date     | 2022-01-05 00:00:00  
 mcap_value    | 5772553.599852260... 
 mcap_rank     | null                 
 created_dt    | 2023-02-14 12:20:... 
 created_by    | farmraider           
 created_proc  | wrangler_assets_e... 
 modified_dt   | 2023-02-21 14:45:... 
 modified_by   | farmraider           
 modified_proc | utils_etl_load_sp... 
 deactivate_dt | null                 
 log_ror       | 0.7292785110966670   
 simp_ror      | 0.0685410000000000   

None


## Replace modify attributes with Nulls
* Set modify_proc & modify_by values to Null
* Drop the modify_dt column

In [4]:
from pyspark.sql import functions as F

_upsert_sdf = _mcap_sdb \
                    .withColumn('modified_by',F.lit(None)) \
                    .withColumn('modified_proc',F.lit(None))
_upsert_sdf = _upsert_sdf.drop(F.col('modified_dt'))
_upsert_sdf.show(n=3,vertical=True)

-RECORD 0-----------------------------
 mcap_past_pk  | 66022                
 uuid          | 6396fb727cc473c58... 
 data_source   | coingecko            
 asset_name    | axial                
 asset_symbol  | axial                
 mcap_date     | 2022-01-05 00:00:00  
 mcap_value    | 5772553.599852260... 
 mcap_rank     | null                 
 created_dt    | 2023-02-14 12:20:... 
 created_by    | farmraider           
 created_proc  | wrangler_assets_e... 
 modified_by   | null                 
 modified_proc | null                 
 deactivate_dt | null                 
 log_ror       | 0.7292785110966670   
 simp_ror      | 0.0685410000000000   



## Upsert table to verify Modify values are auto added]
* Load data again to verify that modify_dt is added and modify_by & modify_proc have values

In [96]:
_tbl_name='warehouse.mcap_past'
_pk = ['mcap_past_pk']
_cols_not_for_update = ['mcap_past_pk','uuid','data_source','asset_name','asset_symbol',
                   'mcap_date','created_dt','created_by','created_proc']
_options={
    "BATCHSIZE":1000,   # batch size to partition the dtaframe
    "PARTITIONS":1,    # number of parallel clusters to run
    "OMITCOLS":_cols_not_for_update,    # columns to be excluded from update
    
}
_records=clsSDB.upsert_sdf_to_table(
    save_sdf=_upsert_sdf,
    db_table=_tbl_name,
    unique_keys=_pk,
    **_options,
)

print("Upserted %d records" % _records)

Validating upsert attributes and parameters ...
Wait a moment, writing data to postgresql tip database ...
Upserted 1 records
